In [10]:
import numpy as np
X = np.array(([2, 9], [1, 5], [3, 6]))
y = np.array(([92], [86], [89]))
y = y/100
def sigmoid(x):
    return 1/(1 + np.exp(-x))
def derivatives_sigmoid(x):
    return x*(1-x)
epoch=10000
lr=0.1
inputlayer_neurons = 2
hiddenlayer_neurons = 3
output_neurons = 1
wh=np.random.uniform(size=(inputlayer_neurons,hiddenlayer_neurons))
bias_hidden=np.random.uniform(size=(1,hiddenlayer_neurons))
weight_hidden=np.random.uniform(size=(hiddenlayer_neurons,output_neurons))
bias_output=np.random.uniform(size=(1,output_neurons))
for i in range(epoch):
    hinp1=np.dot(X,wh)
    hinp= hinp1 + bias_hidden
    hlayer_activation = sigmoid(hinp)

    outinp1=np.dot(hlayer_activation,weight_hidden)
    outinp= outinp1+ bias_output
    output = sigmoid(outinp)

    EO = y-output
    outgrad = derivatives_sigmoid(output)
    d_output = EO * outgrad
    EH = d_output.dot(weight_hidden.T)
    hiddengrad = derivatives_sigmoid(hlayer_activation)
    d_hiddenlayer = EH * hiddengrad
    weight_hidden += hlayer_activation.T.dot(d_output) *lr
    bias_hidden += np.sum(d_hiddenlayer, axis=0,keepdims=True) *lr
    wh += X.T.dot(d_hiddenlayer) *lr
    bias_output += np.sum(d_output, axis=0,keepdims=True) *lr
print("Input: \n" + str(X))
print("Actual Output: \n" + str(y))
print("Predicted Output: \n" ,output)


Input: 
[[2 9]
 [1 5]
 [3 6]]
Actual Output: 
[[0.92]
 [0.86]
 [0.89]]
Predicted Output: 
 [[0.89343996]
 [0.88095496]
 [0.89416436]]


In [6]:
import pandas as pd
from collections import Counter
import math

tennis = pd.read_csv('playtennis.csv')
print("\n Given Play Tennis Data Set:\n\n", tennis)

def entropy(alist):
	c = Counter(x for x in alist)
	instances = len(alist)
	prob = [x / instances for x in c.values()]
	return sum( [-p*math.log(p, 2) for p in prob] )


def information_gain(d, split, target):
	splitting = d.groupby(split)
	n = len(d.index)
	agent = splitting.agg({target : [entropy, lambda x: len(x)/n] })[target]
	agent.columns = ['Entropy', 'observations']
	newentropy = sum( agent['Entropy'] * agent['observations'] )
	oldentropy = entropy(d[target])
	return oldentropy - newentropy


def id3(sub, target, a):
	count = Counter(x for x in sub[target])
	if len(count) == 1:
		return next(iter(count))
	else:
		gain = [information_gain(sub, attr, target) for attr in a]
		print("Gain=",gain)
		maximum = gain.index(max(gain))
		best = a[maximum]
		print("Best Attribute:",best)
		tree = {best:{}}
		remaining = [i for i in a if i != best]
		
		for val, subset in sub.groupby(best):
			subtree = id3(subset,target,remaining)
			tree[best][val] = subtree
		return tree

names = list(tennis.columns)
print("List of Attributes:", names)
names.remove('PlayTennis')
print("Predicting Attributes:", names)
tree = id3(tennis,'PlayTennis',names)
print("\n\nThe Resultant Decision Tree is :\n")
print(tree)


 Given Play Tennis Data Set:

    PlayTennis   Outlook Temperature Humidity    Wind
0          No     Sunny         Hot     High    Weak
1          No     Sunny         Hot     High  Strong
2         Yes  Overcast         Hot     High    Weak
3         Yes      Rain        Mild     High    Weak
4         Yes      Rain        Cool   Normal    Weak
5          No      Rain        Cool   Normal  Strong
6         Yes  Overcast        Cool   Normal  Strong
7          No     Sunny        Mild     High    Weak
8         Yes     Sunny        Cool   Normal    Weak
9         Yes      Rain        Mild   Normal    Weak
10        Yes     Sunny        Mild   Normal  Strong
11        Yes  Overcast        Mild     High  Strong
12        Yes  Overcast         Hot   Normal    Weak
13         No      Rain        Mild     High  Strong
List of Attributes: ['PlayTennis', 'Outlook', 'Temperature', 'Humidity', 'Wind']
Predicting Attributes: ['Outlook', 'Temperature', 'Humidity', 'Wind']
Gain= [0.2467498197744